In [1]:
source('../../source/basic.r')

source('../validataion.r')


Warning message:
“no function found corresponding to methods exports from ‘BSgenome’ for: ‘releaseName’”


In [2]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(Signac)
    library(Seurat)
    library(GenomeInfoDb)
    library(EnsDb.Hsapiens.v86 )# hg38
   # library(EnsDb.Hsapiens.v75) #hg19
    library(ggplot2)
    library(patchwork)
    library(data.table)
    library(Matrix)
    set.seed(1234)
})

# Metadata

In [3]:
metadata.all <- fread('data/tumor_metadata_Full_Cohort.tsv')

In [4]:
metadata.all%>%dim

[1] 74621    20

In [5]:
colnames(metadata.all)

[1] "Barcode"              "Sample"               "TSSEnrichment"       
 [4] "ReadsInTSS"           "ReadsInPromoter"      "ReadsInBlacklist"    
 [7] "PromoterRatio"        "PassQC"               "NucleosomeRatio"     
[10] "nMultiFrags"          "nMonoFrags"           "nFrags"              
[13] "nDiFrags"             "DoubletScore"         "DoubletEnrichment"   
[16] "BlacklistRatio"       "ATAC_clusters"        "predictedCell_ArchR" 
[19] "predictedGroup_ArchR" "predictedScore_ArchR"

In [6]:
metadata.all$Sample%>%unique

[1] "3533EL" "3571DL" "36186L" "36639L" "366C5L" "37EACL" "38FE7L" "3BAE2L"
 [9] "3CCF1L" "3E4D1L" "3E5CFL"

In [7]:
metadata.rna <- fread('data/tumor_metadata_Full_Cohort_mmc3.tsv')#%>%dplyr::filter(CNV.Pos==TRUE)

# cpeak data

## read data

In [8]:
sample_names = metadata.all$Sample%>%unique

In [9]:
sample_names

[1] "3533EL" "3571DL" "36186L" "36639L" "366C5L" "37EACL" "38FE7L" "3BAE2L"
 [9] "3CCF1L" "3E4D1L" "3E5CFL"

In [10]:
# sample_name = sample_names[1]

In [5]:
sample_name='3BAE2L'

In [25]:
fun_seurat <- function(sample_name){

    suppressMessages({
        ################################
        ## ATAC part
        ## list the dir
        dir_mtx = list.files('data/peak_calling/',pattern = 'gz.mtx')%>%.[grep(sample_name,.)]
        frag_dir = list.files('data/peak_calling/',pattern = 'tsv.gz$')%>%.[grep(sample_name,.)]
        peak_dir = list.files('data/peak_calling/',pattern = 'narrowPeak.bed$')%>%.[grep(sample_name,.)]

        counts <- readMM(paste0("data/peak_calling/",dir_mtx)) 
        colnames(counts) <- fread(paste0('data/barcode/',sample_name,'.barcode'),header = FALSE)%>%
                            pull(V1)
        rownames(counts) <- fread(paste0('data/peak_calling/',peak_dir))%>%mutate(name=paste0(V1,':',V2,'-',V3))%>%pull(name)
        
        ## read metadata
        metadata = metadata.all%>%dplyr::filter(Sample==sample_name)%>%
                mutate(Barcode=str_extract(Barcode,'(?<=#).*'))%>%
                column_to_rownames('Barcode')

        ## subset using metadata
        counts.sub <- counts[,intersect(rownames(metadata),colnames(counts))]
        metadata.sub <- metadata[intersect(rownames(metadata),colnames(counts)),]

        ## build seurat object
        chrom_assay <- CreateChromatinAssay(
          counts = counts.sub,
          sep = c(":", "-"),
          genome = 'hg38',
          fragments = paste0('./data/',frag_dir),
          min.cells = 0,
          min.features = 0
        )
        seurat.object.all <- CreateSeuratObject(
          counts = chrom_assay,
          assay = "peaks",
          meta.data = metadata.sub
        )
        seurat.object <- subset(seurat.object.all, cells = rownames(metadata.sub))

        ## Normalize & TF-IDF & SVD
        seurat.object <- RunTFIDF(seurat.object)
        seurat.object <- FindTopFeatures(seurat.object, min.cutoff = 'q0')
        seurat.object <- RunSVD(seurat.object)
        p.cor <- DepthCor(seurat.object)

        ## Clustering &UMAP


            seurat.object <- RunUMAP(object = seurat.object, reduction = 'lsi', dims = 2:30)
            seurat.object <- FindNeighbors(object = seurat.object, reduction = 'lsi', dims = 2:30)
            seurat.object <- FindClusters(object = seurat.object, verbose = FALSE, algorithm = 3)


        ## plot
        p.umap.cluster = DimPlot(object = seurat.object, label = TRUE) + theme(aspect.ratio = 1)

        # only plot large clusters
        cells.sub <- metadata.sub%>%rownames_to_column('cell')%>%group_by(predictedGroup_ArchR)%>%
                    mutate(n=n())%>%ungroup%>% dplyr::filter(n>100)%>%pull(cell)
        p.umap.label = DimPlot(object = seurat.object,cells = cells.sub, group.by='predictedGroup_ArchR',label = TRUE) + 
                        theme(aspect.ratio = 1) 

        ## add DNA metadata
        metadata.all <- metadata%>%rownames_to_column('ID')%>%
                        left_join(metadata.rna,by=c('predictedCell_ArchR'='Barcode'))%>%column_to_rownames('ID')

        seurat.object@meta.data <-cbind(seurat.object@meta.data,total.cnv=metadata.all$Total_CNVs)
        seurat.object@meta.data <-cbind(seurat.object@meta.data,cnv=metadata.all$CNV.Pos)


        p.umap.cnv = FeaturePlot(object = seurat.object,features='total.cnv',label = FALSE) + theme(aspect.ratio = 1)  

        ######################################
        ## ATAC gene activity

        # extract gene annotations from EnsDb
        annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86 )



        # # change to UCSC style since the data was mapped to hg19
        seqlevelsStyle(annotations) <- 'UCSC'

        DefaultAssay(seurat.object) <- "peaks"

        # add the gene information to the object
        Annotation(seurat.object) <- annotations

        gene.activities <- GeneActivity(seurat.object)

        # add the gene activity matrix to the Seurat object as a new assay and normalize it
        seurat.object[['RNA']] <- CreateAssayObject(counts = gene.activities)
        seurat.object <- NormalizeData(
          object = seurat.object,
          assay = 'RNA',
          normalization.method = 'LogNormalize',
          scale.factor = median(seurat.object$nCount_RNA)
        )

        ###################################
        # RNA part

        ## read data
        dir_mtx = list.files('data/rna/',pattern = 'mtx.gz')%>%.[grep(sample_name,.)]
        dir_barcode = list.files('data/rna/',pattern = 'barcodes')%>%.[grep(sample_name,.)]
        dir_feature = list.files('data/rna/',pattern = 'features')%>%.[grep(sample_name,.)]
        counts <- readMM(paste0("data/rna/",dir_mtx)) 
        colnames(counts) <- read_table(paste0("data/rna/",dir_barcode),col_names  = FALSE,col_types = cols())%>%
                            pull(X1)
        rownames(counts) <- read_table(paste0("data/rna/",dir_feature),col_names  = FALSE,col_types = cols())%>%pull(X2)%>%make.unique

        ## seurat object
        metadata.rna.sub <- metadata.rna%>%dplyr::filter(Sample==sample_name)%>%
                            mutate(Barcode=gsub('_.$|_..$','',Barcode))%>%column_to_rownames('Barcode')
        seurat.rna <- CreateSeuratObject(counts = counts, project = "tumor",meta.data = metadata.rna.sub)
        seurat.rna <- subset(seurat.rna, cells = rownames(metadata.rna.sub))

        ## normalize  & scale
        seurat.rna <- NormalizeData(seurat.rna, normalization.method = "LogNormalize", scale.factor = 10000)

        seurat.rna <- FindVariableFeatures(seurat.rna, selection.method = "vst", nfeatures = 2000)

        all.genes <- rownames(seurat.rna)
        seurat.rna <- ScaleData(seurat.rna, features = all.genes)

        ## PCA & UMAP & clustering
        seurat.rna <- RunPCA(seurat.rna, features = VariableFeatures(object = seurat.rna))
        seurat.rna <- RunUMAP(seurat.rna, dims = 1:20)
        seurat.rna <- FindNeighbors(seurat.rna, dims = 1:20)
        seurat.rna <- FindClusters(seurat.rna, resolution = 0.5)

        # plots

        # note that you can set `label = TRUE` or use the LabelClusters function to help label
        # individual clusters
        p.umap.cluster.rna <- DimPlot(seurat.rna, reduction = "umap")+theme(aspect.ratio = 1)

        # note that you can set `label = TRUE` or use the LabelClusters function to help label
        # individual clusters
        p.umap.label.rna <- DimPlot(seurat.rna,group.by = 'cell.type', reduction = "umap",label = TRUE)+theme(aspect.ratio = 1)

        p.umap.cnv.rna <- FeaturePlot(object = seurat.rna,features='Total_CNVs',label = FALSE) +theme(aspect.ratio = 1)

        ###########################################################
        # link RNA ATAC

        DefaultAssay(seurat.object) <- 'RNA'

        transfer.anchors <- FindTransferAnchors(
          reference = seurat.rna,
          query = seurat.object,
          reduction = 'cca'
        )

        predicted.labels <- TransferData(
          anchorset = transfer.anchors,
          refdata = seurat.rna$cell.type,
          weight.reduction = seurat.object[['lsi']],
          dims = 2:30
        )
        seurat.object <- AddMetaData(object = seurat.object, metadata = predicted.labels)


        plot1 <- DimPlot(
          object = seurat.rna,
          group.by = 'cell.type',
          label = TRUE,
          repel = TRUE) + NoLegend() + ggtitle('scRNA-seq')

        plot2 <- DimPlot(
          object = seurat.object,
          group.by = 'predicted.id',
          label = TRUE,
          repel = TRUE) + NoLegend() + ggtitle('scATAC-seq')

        plot.join <- (plot1+theme(aspect.ratio = 1)) + ((plot2+theme(aspect.ratio = 1)))
    })
    ## results
    res <- list(metadata.sub = metadata.sub,
                seurat.object=list(atac=seurat.object,rna=seurat.rna), 
                plot=list(p.umap.cluster=p.umap.cluster,
                          p.umap.label=p.umap.label,
                          p.umap.cnv=p.umap.cnv,
                          p.umap.cluster.rna=p.umap.cluster.rna,
                          p.umap.label.rna = p.umap.label.rna,
                          p.umap.cnv.rna=p.umap.cnv.rna,
                          p.join = plot.join))
    ## save all plots
    pdf(paste0('plot/peaks/',sample_name,'.pdf'),height = 6,width=16)
    (res$plot)%>%print
    dev.off()
    
    qsave(res,paste0('rds/peaks/',sample_name,'.qs'))
    
    return(res)
        
}

In [26]:
#res.list <- mclapply(sample_names,fun_seurat,mc.cores = 10)

In [27]:
#tmp <- res.list

In [28]:
res.list <- c()

In [ ]:
for(i in sample_names){
    
    res.list[[i]] <- fun_seurat(i)
    
}

In [30]:
res.list%>%qsave('rds/peaks/all.list.qs')

In [32]:
res.list.extend <- c()

In [ ]:
for (i in sample_names){
    
    seurat.object <- res.list[[i]]$seurat.object$atac
    seurat.rna <- res.list[[i]]$seurat.object$rna

    tmp <- qread('metadata_trans.qs')
    tmp.meta = seurat.object@meta.data 

    metadata.expand <- seurat.object@meta.data%>%rownames_to_column('a')%>%mutate(clusterID=str_extract(predictedGroup_ArchR,'.*(?=-)')%>%as.numeric,
                                    clusterType=str_extract(predictedGroup_ArchR,'(?<=-).*'))%>%
                        left_join(tmp,by=c('clusterID'))%>%column_to_rownames('a')

    seurat.object@meta.data <- metadata.expand

    p.umap.label3 = DimPlot(object = seurat.object, group.by='celltype',label = TRUE) + 
                    theme(aspect.ratio = 1) 

    p.umap.label4 = DimPlot(object = seurat.object,group.by='clusterID',label = TRUE) + 
                    theme(aspect.ratio = 1) 

    DefaultAssay(seurat.object) <- 'RNA'
    p.feature.atac = FeaturePlot(seurat.object, features = c('KIT','WFDC2','MUC16'),ncol = 3)#+NoLegend()

    p.feature.rna = FeaturePlot(seurat.rna, features = c('KIT','WFDC2','MUC16'),ncol = 3)#+theme(aspect.ratio = 1)

    
    res <- list(seurat.object=list(atac=seurat.object,rna=seurat.rna), 
            plot=list(p.umap.cluster=res.list[[i]]$plot$p.umap.cluster,
                      p.umap.label=res.list[[i]]$plot$p.umap.label,
                      p.umap.label3=p.umap.label3,
                      p.umap.label4=p.umap.label4,
                      p.umap.cnv=res.list[[i]]$plot$p.umap.cnv,
                      p.umap.cluster.rna=res.list[[i]]$plot$p.umap.cluster.rna,
                      p.umap.label.rna = res.list[[i]]$plot$p.umap.label.rna,
                      p.umap.cnv.rna=res.list[[i]]$plot$p.umap.cnv.rna,
                      p.feature.atac=p.feature.atac,
                      p.feature.rna=p.feature.rna,
                      p.join = res.list[[i]]$plot$plot.join))
     
    ## save all plots
    pdf(paste0('plot/peaks/',i,'_extend.pdf'),height = 6,width=18)
    (res$plot)%>%print
    dev.off()
    
     qsave(res,paste0('rds/peaks/',i,'_extend.qs'))
     res.list.extend[[i]] <- res
}


In [35]:
res.list.extend%>%qsave('rds/cpeaks/all.list.extend.qs')